### ✅ Pattern ปกติที่ใช้กันในงานจริง (สำหรับ PySpark + Big Data)

- โดยเฉพาะในงาน Data Engineering หรือ Data Analytics ที่ใช้ Spark 
  เช่น วิเคราะห์ log, transaction, sales, หรือ IoT data

#### 🔸 สรุป Step ที่นิยมใช้เป็น Pattern (เรียงตามลำดับที่ทำจริง):

##### 1.Inspect Schema & Sample

- `df.printSchema()`, `df.show()`, `df.count()`

- ✔ ดูว่า column เป็น type อะไร และข้อมูลมีหน้าตาแบบไหน

##### 2.Check Missing / Null

- `count(when(col.isNull(), col))` หรือ `isnan`

- ✔ ใช้บ่อยมาก เพื่อดูว่าสามารถนำไปใช้งานได้เลยไหม หรือต้อง clean

##### 3.Summary Stats (เฉพาะ numeric column)

- `df.describe().show()`

- ✔ ใช้บ่อย เพื่อดูค่าเฉลี่ย/Max/Min เร็วๆ

✅ แนวทางการเลือกดู Summary Stats (อย่างมีหลักการ)

🔸 1. ดูเฉพาะ numeric columns ที่เกี่ยวข้องกับ use case / ธุรกิจ

| Column        | ประเภท | ควรดูไหม?  | เหตุผล                                        |
| ------------- | ------ | ---------- | --------------------------------------------- |
| `price`       | float  | ✅          | ตรวจสอบราคา: ค่าเฉลี่ย, ค่าสูงสุดผิดปกติ      |
| `quantity`    | int    | ✅          | ดูจำนวนหนังสือที่ขายต่อรายการ                 |
| `total_sales` | float  | ✅          | เช็กยอดขายรวม                                 |
| `stock`       | int    | ✅ ถ้าใช้   | ดูของคงคลัง มี outlier หรือของหมดไหม          |
| `rating`      | float  | ✅ ถ้ามี    | ดูแนวโน้มรีวิว                                |
| `year`        | int    | 🔶 ไม่เสมอ | บางครั้งต้องแปลงเป็น datetime ก่อนจึงดูค่าได้ |


🔸 2. ถ้า dataset ยังไม่คุ้นเคย —> ดู summary ของ numeric ทุกตัว (รอบแรก)

In [ ]:
df.select([col for col in df.columns if dict(df.dtypes)[col] in ['int', 'double', 'float']]).describe().show() # type: ignore

🔸 3. จากนั้นค่อย “โฟกัสเจาะลึก” เฉพาะ column ที่:

- ค่าผิดปกติ (เช่น price ต่ำกว่า 0 หรือสูงเกินจริง)

- กระจายสูง (stddev สูง) เช่น คนซื้อ 1 กับ 100 เล่ม

- ค่ามี missing (count ต่ำกว่าจำนวนแถวทั้งหมด)

- เกี่ยวข้องกับเป้าหมายของโปรเจกต์ (KPI สำคัญ)

🎯 หลักคิด: Summary Stats ไม่ใช่แค่เช็กว่า "ปกติไหม" แต่คือ “ตั้งคำถาม”

| Metric     | คำถาม                                                    |
| ---------- | -------------------------------------------------------- |
| mean       | ค่ากลางสมเหตุสมผลไหม เช่น ราคาเฉลี่ย 100 บาท โอเคไหม?    |
| stddev     | ความหลากหลายเยอะไหม? ถ้าเยอะ ทำไม?                       |
| min/max    | มีค่าแปลก ๆ ไหม เช่น quantity = 0, price = -50           |
| count      | มี missing value หรือเปล่า? ควรจัดการยังไง?              |
| percentile | มีกลุ่มที่ขายดีมาก ๆ อยู่แค่ 5% ไหม? (long-tail effect?) |

- 1.`count`
  
  - ใช้ดูว่า มี missing value ไหม (ถ้า count น้อยกว่าจำนวนแถวทั้งหมด แปลว่ามี null)

- 2.`mean` (ค่าเฉลี่ย)
  
  - ใช้ดูว่าข้อมูล "โดยรวม" อยู่ที่ระดับไหน

- 3.`stddev` ส่วนเบี่ยงเบนมาตรฐาน
  
  - ใช้ดูว่าข้อมูลกระจายมากแค่ไหน
  
  - ถ้า `stddev` ต่ำ → ข้อมูลใกล้เคียงกัน
  
  - ถ้า `stddev` สูง → มีข้อมูลหลุด หรือกระจายกว้าง
  
  - ใช้ตรวจ outlier ด้วยสูตร: ค่าปกติ = mean ± 3 * stddev อะไรที่หลุดจากนี้ → สงสัยว่าอาจผิดปกติ

- 4.`min` กับ `max`
  
  -ใช้ดูว่า ช่วงข้อมูลกว้างไหม หรือมี outlier แบบสุดโต่งหรือไม่

  - ถ้า `min` = 0 หรือ `max` สูงผิดปกติ → ตรวจต่อว่าเป็นข้อผิดพลาดหรือ outlier จริงไหม

✅สรุป: สิ่งที่ควรตรวจหลังใช้ describe

| เป้าหมาย             | ดูคอลัมน์ไหน            | วิธีคิด                                      |
| -------------------- | ----------------------- | -------------------------------------------- |
| Missing value        | `count`                 | น้อยกว่าจำนวนแถว?                            |
| ค่าผิดปกติ (Outlier) | `mean`, `stddev`, `max` | เกิน `mean ± 3*stddev` ไหม?                  |
| ช่วงข้อมูลกว้างไหม   | `min`, `max`            | ห่างจาก mean มากไหม?                         |
| การกระจายข้อมูล      | `stddev`                | สูง → ข้อมูลหลากหลาย, อาจต้องจัดการเพิ่มเติม |


✨ สรุปสั้น ๆ

- ✅ ดูเฉพาะ numeric columns ที่เกี่ยวข้องกับธุรกิจ หรือใช้วิเคราะห์จริง

- 🧠 ตั้งคำถามจากค่าเหล่านั้น ไม่ใช่แค่ดูผ่าน ๆ

- 🔁 รอบแรกอาจดูทั้งหมด แล้วค่อยโฟกัสลงในจุดสำคัญ

##### 4.Value Count / Distinct / Top-N

- `groupBy().count()` / `distinct().count()`

- ✔ สำคัญมากเวลาต้องการดู category ที่มีอยู่จริง เช่น genre, book title

##### 5.Trend Analysis (กรณีมีเวลา/วันที่)

- `groupBy('month')` หรือ `year-month` แล้ว sum ยอดขาย

- ✔ ถ้าข้อมูลมีเวลา เกือบทุกงานต้องดู trend รายเดือนหรือรายวัน

##### 6.Distribution / Visualization (optional)

- ใช้ `toPandas().sample()` เพื่อ plot histogram, bar chart, correlation

- ✔ ใช้เฉพาะตอนมีเวลาหรือในงาน presentation / dashboard

##### 7.Correlation Analysis (optional)

- ✔ ใช้ในกรณีที่วิเคราะห์เชิงลึก หรือเตรียมข้อมูลไปทำ model

##### 8.Outlier / Invalid Detection (กรณีต้อง clean data)

- ✔ เช่น quantity ≤ 0, price < 0 — ใช้เฉพาะตอนรู้ว่าค่าพวกนี้ผิดแน่

##### 9.Data Cleaning / Save cleaned Data

- ✔ ขั้นตอนสุดท้ายก่อนส่งต่อข้อมูล

#### 🧠 ถ้าสรุปเป็น Core EDA Pattern ที่ควรทำ "แทบทุกครั้ง"

| ขั้นตอน             | ความจำเป็น     | ทำไปทำไม                               |
| ------------------- | -------------- | -------------------------------------- |
| Schema inspection   | 🔴 สูง         | รู้ว่า column มีอะไรและ type ถูกไหม    |
| Missing/null check  | 🔴 สูง         | ข้อมูลพร้อมใช้ไหม?                     |
| Summary stats       | 🟠 กลาง        | เข้าใจภาพรวมค่า numeric                |
| Value count / group | 🔴 สูง         | เข้าใจ distribution ของแต่ละ category  |
| Trend / Time Series | 🟠 ถ้ามีวันที่ | หาความเปลี่ยนแปลงตามเวลา               |
| Visualization       | 🟢 เสริม       | ทำให้เห็นภาพง่าย ใช้ใน notebook/นำเสนอ |
| Correlation         | 🟢 เสริม       | ถ้าจะ model ต่อ                        |
| Outlier check       | 🟠 กลาง        | ถ้าจะ clean ข้อมูล                     |
| Save clean data     | 🔴 สูง         | เพื่อใช้ขั้นตอนต่อไป                   |


##### 📌 สรุป :

- ไม่ต้องทำครบทุกข้อทุกครั้งแต่ควรทำ 4 อย่างนี้ให้ได้เกือบทุกงาน:

- ✅ printSchema()

- ✅ Missing/null check

- ✅ describe()

- ✅ GroupBy & Aggregate เพื่อดู distribution (Top-N หรือ Sum ยอดขาย)

#### การจัดการกับข้อมูลที่เป็น `Null` และ `Outlier`

#### 🧱 1. กรณีข้อมูลที่เป็น null (Missing Data)

✅ แนวทางการจัดการ:

| วิธีจัดการ                                           | เหมาะกับกรณี                                 | ตัวอย่าง                             |
| ---------------------------------------------------- | -------------------------------------------- | ------------------------------------ |
| ลบทิ้ง (Drop rows)                                   | ถ้ามีจำนวนน้อย และไม่สำคัญ                   | ข้อมูลสูญหายแค่ 1% ของแถว            |
| เติมด้วยค่าเฉลี่ย/มัธยฐาน (`mean` / `median`)        | ถ้าเป็น **ตัวเลข** และค่ากระจายไม่มาก        | เติม `mean(total_price)`             |
| เติมด้วยค่าที่กำหนดไว้ (`0`, `"Unknown"` ฯลฯ)        | ถ้าเป็น **categorical** หรือ logical default | `customer_type = Unknown`            |
| เติมด้วยค่าก่อนหน้า (`ffill`) หรือค่าถัดไป (`bfill`) | ข้อมูลตามลำดับเวลา (time-series)             | เติมอุณหภูมิจากวันก่อน               |
| ใช้ model ทำนายค่า missing                           | งานวิเคราะห์ขั้นสูง                          | ใช้ regression เติม `price` ที่หายไป |


✅ Tips: ก่อนตัดสินใจ ให้ดู count() กับ percent missing เสมอ

##### ✅ วิธีจัดการกับ Missing Values แบ่งตามกรณี

🔹 1. สำหรับงาน Dashboard / BI / Reporting

| กรณี                                                                     | แนวทางที่นิยม                             |
| ------------------------------------------------------------------------ | ----------------------------------------- |
| ข้อมูลขาดเพราะ **ไม่จำเป็นต้องกรอกจริง** (เช่น ลูกค้าที่ไม่กรอกเบอร์โทร) | ✅ แสดงว่า missing ได้ ไม่ต้องเติม         |
| ข้อมูลขาดแบบ **ผิดพลาด** (ควรมีแต่ไม่มี เช่น `price = null`)             | 🔧 เติม (impute) หรือกรองออก              |
| ข้อมูลสำคัญ **ขาดเยอะมาก**                                               | ⚠️ พิจารณา **ดรอปคอลัมน์** เลยถ้าไม่สำคัญ |


💡 เป้าหมาย: ให้ข้อมูลสมบูรณ์พอจะนำไปสรุปภาพรวมและสร้างกราฟได้ โดยไม่หลอกตาผู้ใช้งาน

🔹 2. สำหรับ Machine Learning

| กรณี                                            | แนวทางที่นิยม                                                     |
| ----------------------------------------------- | ----------------------------------------------------------------- |
| Column สำคัญแต่ขาดค่า (missing ไม่มาก)          | ✅ เติมด้วย mean / median / mode (หรือค่าที่เหมาะกับ distribution) |
| Missing มีนัยยะ เช่น "ไม่ได้กรอก" อาจเป็นสัญญาณ | ✅ สร้าง flag column เช่น `is_missing = true`                      |
| Missing เยอะมากจนไม่มีประโยชน์                  | ❌ ดรอป column นั้น                                                |


💡 เป้าหมาย: ไม่ให้โมเดลเจอ null เพราะอัลกอริธึมส่วนใหญ่ไม่รองรับ ต้องเติมให้ครบหรือแจ้งความหมาย

##### 🔧 เทคนิคยอดนิยมในการจัดการ Missing Values

| เทคนิค                                | ใช้เมื่อ                                    |
| ------------------------------------- | ------------------------------------------- |
| เติมค่าเฉลี่ย (`mean`)                | ค่าเป็นตัวเลขกระจายไม่มาก                   |
| เติมค่ากลาง (`median`)                | ตัวเลขมี outlier เยอะ                       |
| เติมค่าที่พบบ่อย (`mode`)             | ข้อมูลเชิง category เช่น จังหวัด, เพศ       |
| เติมด้วย "unknown"                    | กรณีไม่แน่ใจ ให้กรอก “unknown” แทน null     |
| เติมด้วยค่าพิเศษ เช่น `-999` หรือ `0` | ใช้กับ ML (และต้องสร้าง flag `was_missing`) |
| ลบทิ้ง (`drop`)                       | ข้อมูลมีไม่มากหรือ column นั้นไม่จำเป็น     |


🧠 สรุปง่ายๆ

| งาน                | แนวทางจัดการ Missing                                                       |
| ------------------ | -------------------------------------------------------------------------- |
| **Dashboard / BI** | เติมเฉพาะจุดที่มีผลต่อการแสดงผล (mean, median, mode), หรือกรองออก          |
| **ML**             | เติมให้ครบทุก row, สร้าง flag `was_missing`, หรือ drop column ที่ไม่จำเป็น |


#### 🧨 2. กรณีข้อมูลที่เป็น outlier

- เมื่อเราใช้ Describe Summary Statistics แล้วเราต้องตรวจสอบหา Outlier จาก IQR หรือ Z-score

##### 🔹 สองวิธีหลักในการตรวจสอบ Outlier

1. IQR Method (Interquartile Range) → ใช้ในทางสถิติทั่วไป ( นิยม )
- คำนวณจาก:
  
  - IQR = Q3 - Q1
  
  - Outlier = ค่าใดๆ ที่อยู่นอกช่วง
  
  - Q1−1.5×IQR,Q3+1.5×IQR
  
  - ใช้กับข้อมูลที่อาจไม่ปกติ (non-normal)

  เหมาะกับ: ข้อมูลจริงในธุรกิจ เช่น ยอดขาย, รายได้, จำนวนวันใช้งาน ฯลฯ นิยมใช้ใน EDA, BI, และ dashboard

2. Z-score Method (Mean ± 3×StdDev) → ใช้กับข้อมูลที่เป็น Normal
- คำนวณจาก:
  
  - ค่า Z-score = (x-mean)/stddev
  
  - Outlier = ค่าใดๆ ที่มี |Z| > 3
  
  - ต้องแน่ใจว่าข้อมูลใกล้เคียง normal จริง (ใช้ histogram, skewness เช็คได้)

  เหมาะกับ: ข้อมูลเชิงทดลอง, sensor, ML preprocessing ฯลฯ

📌 สรุปเปรียบเทียบ

| วิธี    | ต้องเป็น Normal? | ใช้บ่อยกับ    | เกณฑ์ตัด   | ความนิยม        |
| ------- | ---------------- | ------------- | ---------- | --------------- |
| IQR     | ❌ ไม่ต้อง        | Business, EDA | Q1±1.5×IQR | ✅ มาก           |
| Z-score | ✅ ต้อง           | ML, Math      | mean±3×std | 🔄 ใช้เฉพาะกรณี |

##### เมื่อเจอ outlier ต้องจัดการยังไง?

✅ แนวทางการจัดการ:

| วิธีจัดการ                               | เหมาะกับกรณี               | ตัวอย่าง                        |
| ---------------------------------------- | -------------------------- | ------------------------------- |
| ลบทิ้ง                                   | ถ้าเป็นค่าผิดพลาดที่ชัดเจน | ราคา = 999999 ในสินค้าธรรมดา    |
| ตัดค่าให้เป็น upper/lower limit (`clip`) | ถ้าค่าหลุดนิดหน่อย         | limit `quantity` ให้อยู่ 1–10   |
| แปลงข้อมูล (Transformation)              | ถ้าข้อมูล skew มาก         | ใช้ `log(price)` ลดผล outlier   |
| สร้าง feature ใหม่แทน (แทนที่จะลบ)       | สำหรับ ML model            | เพิ่ม `is_outlier` เป็น feature |
| เก็บไว้หาความรู้เชิงธุรกิจ               | ถ้าข้อมูลดู “จริง”         | ลูกค้าสั่งซื้อ 1000 ชิ้นจริง ๆ  |

✅ Tips: อย่าลบ outlier ถ้า อาจเป็น insight สำคัญทางธุรกิจ เช่น ลูกค้ารายใหญ่

##### ✅ วิธีจัดการ Outlier แบ่งตามวัตถุประสงค์ของงาน

🔹 1. ถ้าทำ Dashboard / BI / Report

| กรณี                                                    | แนวทางที่นิยม                                       |
| ------------------------------------------------------- | --------------------------------------------------- |
| **Outlier ที่ผิดพลาดจริง (เช่น 9999, -100)**            | ลบทิ้งหรือตัดออก                                    |
| **Outlier ที่เป็นจริง (เช่น VIP customer ซื้อเยอะมาก)** | ✅ เก็บไว้แยกกลุ่ม เช่น segment หรือ plot แยกต่างหาก |


💡 เหตุผล: เพื่อไม่ให้ค่าเฉลี่ย/กราฟโดนลากผิดเพี้ยน เช่น bar chart มี scale สูงเกินจาก outlier เดียว

🔹 2. ถ้าใช้กับ Machine Learning Model

| กรณี                       | แนวทางที่นิยม                                                                          |
| -------------------------- | -------------------------------------------------------------------------------------- |
| **Outlier ที่ผิดพลาดจริง** | แก้ไขหรือลบทิ้ง (เพราะมันรบกวนโมเดล)                                                   |
| **Outlier ที่เป็นจริง**    | ✅ *ไม่ลบ!* เก็บไว้และ **สร้าง feature** เช่น `is_outlier = true` เพื่อให้โมเดลเรียนรู้ |

💡 เหตุผล: โมเดลจะเรียนรู้จากพฤติกรรมของกลุ่ม outlier ได้ เช่น ลูกค้าระดับสูง, ผู้ใช้ผิดปกติ ฯลฯ

✳️ เทคนิคเสริมที่ใช้ร่วม:

- สร้าง is_outlier = true/false ไว้ใน dataset

- อาจใช้เทคนิค Robust Model หรือ Log Transform ลดผลกระทบของ outlier

- ใน unsupervised (เช่น clustering) ก็ใช้ outlier เป็น insight ได้

🎯 สรุปอีกทีแบบย่อ:

| งาน              | ทำกับ Outlier ยังไง                                               |
| ---------------- | ----------------------------------------------------------------- |
| **BI/Dashboard** | ลบ outlier ที่ผิด, แยกแสดง outlier ที่จริง                        |
| **ML**           | ลบแค่ที่ผิดพลาด, ที่จริงให้เก็บไว้ พร้อมสร้าง flag (`is_outlier`) |

##### 🛠 ตัวอย่างการจัดการใน PySpark

👉 ลบแถวที่มี null:

In [ ]:
df_clean = df.dropna(subset=["total_price", "quantity"]) # type: ignore

👉 เติมค่าด้วย mean:

In [ ]:
from pyspark.sql.functions import mean
mean_price = df.select(mean("total_price")).collect()[0][0] # type: ignore
df_filled = df.na.fill({"total_price": mean_price}) # type: ignore


👉 ลบ outlier (เกิน ±3 stddev):

In [ ]:
from pyspark.sql.functions import col, mean, stddev

stats = df.select(mean("total_price"), stddev("total_price")).first() # type: ignore
mean_val = stats[0]
std_val = stats[1]

df_filtered = df.filter( # type: ignore
    (col("total_price") >= mean_val - 3 * std_val) &
    (col("total_price") <= mean_val + 3 * std_val)
)

#### 🧠 สรุปเป็น Pattern ที่ใช้จริง

| กรณี                        | สิ่งที่ทำ                                      |
| --------------------------- | ---------------------------------------------- |
| NULL < 5%                   | ลบทิ้ง                                         |
| NULL > 30%                   | เติม mean/median หรือ "Unknown"                |
| NULL มี logic อื่นให้เติม   | คำนวณจาก column อื่น                           |
| Outlier จริง                | แยกไว้, สร้าง flag                             |
| Outlier ผิด                 | ลบหรือ fix                                     |
| ไม่แน่ใจว่า outlier จริงไหม | ตรวจสอบกับธุรกิจหรือ log ระบบ                  |
| ส่งต่อไปใช้ ML              | สร้าง column ใหม่ เช่น `is_outlier`, `is_null` |
